In [1]:
#!pip install timesfm

In [ ]:
import timesfm

In [ ]:
import pandas as pd
data=pd.read_csv("Electric_Production.csv")
data.head()

In [ ]:
data["DATE"]=pd.to_datetime(data["DATE"])
data.head()

In [ ]:
df = pd.DataFrame({"unique_id": [1] * len(data), "ds": data["DATE"], "y":data["IPG2211A2N"]})
df.head(2)

In [ ]:
len(df)

In [ ]:
split_idx = int(len(df) * 0.94)
train_df = df[:split_idx]
test_df = df[split_idx:]
print(train_df.shape, test_df.shape)

In [9]:
tfm = timesfm.TimesFm(
   context_len=512,       # Length of the context window for the model
   horizon_len=24,        # Forecasting horizon length
   input_patch_len=32,    # Length of input patches
   output_patch_len=128,  # Length of output patches
   num_layers=20,
   model_dims=1280,
   backend="cpu"
)

In [ ]:
tfm.load_from_checkpoint(repo_id="google/timesfm-1.0-200m")

In [ ]:
timesfm_forecast = tfm.forecast_on_df(
   inputs=train_df,       # Input training data for training
   freq="MS",             # Frequency of the time-series data
   value_name="y",        # Name of the column containing the values to be forecasted
   num_jobs=-1            # Set to -1 to use all available cores
)
timesfm_forecast.head()

In [ ]:
timesfm_forecast = timesfm_forecast[["ds","timesfm"]]
timesfm_forecast.head()

In [13]:
import numpy as np

def mse(y_pred, y_true):
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    return np.mean(np.square(y_pred - y_true))

def mae(y_pred, y_true):
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    return np.mean(np.abs(y_pred - y_true))

def rmse(y_pred, y_true):
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

In [ ]:
len(timesfm_forecast["timesfm"]), len(test_df["y"])

In [ ]:
print(f"Mean Absolute Error (MAE): {mae(timesfm_forecast["timesfm"], test_df["y"])}")
print(f"Mean Squared Error (MSE): {mse(timesfm_forecast["timesfm"], test_df["y"])}")
print(f"Root Mean Squared Error (RMSE): {rmse(timesfm_forecast["timesfm"], test_df["y"])}")

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(9, 5))
plt.plot(df["ds"], df["y"], label="Actual")
plt.plot(timesfm_forecast["ds"], timesfm_forecast["timesfm"], label="Predicted")
plt.ylabel("Electric Production")
plt.xlabel("Date")
plt.legend()
plt.show()

# Covariates

Let"s take a toy example of forecasting sales for a grocery store:

**Task:** Given the observed the daily sales of this week (7 days), forecast the daily sales of next week (7 days).

```
Product: ice cream
Daily_sales: [30, 30, 4, 5, 7, 8, 10]
Category: food
Base_price: 1.99
Weekday: [0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6]
Has_promotion: [Yes, Yes, No, No, No, Yes, Yes, No, No, No, No, No, No, No]
Daily_temperature: [31.0, 24.3, 19.4, 26.2, 24.6, 30.0, 31.1, 32.4, 30.9, 26.0, 25.0, 27.8, 29.5, 31.2]
```

```
Product: sunscreen
Daily_sales: [5, 7, 12, 13, 5, 6, 10]
Category: skin product
Base_price: 29.99
Weekday: [0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6]
Has_promotion: [No, No, Yes, Yes, No, No, No, Yes, Yes, Yes, Yes, Yes, Yes, Yes]
Daily_temperature: [31.0, 24.3, 19.4, 26.2, 24.6, 30.0, 31.1, 32.4, 30.9, 26.0, 25.0, 27.8, 29.5, 31.2]
```

In this example, besides the `Daily_sales`, we also have covariates `Category`, `Base_price`, `Weekday`, `Has_promotion`, `Daily_temperature`. Let"s introduce some concepts:

**Static covariates** are covariates for each time series.
- In our example, `Category` is a **static categorical covariate**,
- `Base_price` is a **static numerical covariates**.

**Dynamic covariates** are covaraites for each time stamps.
- Date / time related features can be usually treated as dynamic covariates.
- In our example, `Weekday` and `Has_promotion` are **dynamic categorical covariates**.
- `Daily_temperate` is a **dynamic numerical covariate**.

**Notice:** Here we make it mandatory that the dynamic covariates need to cover both the forecasting context and horizon. For example, all dynamic covariates in the example have 14 values: the first 7 correspond to the observed 7 days, and the last 7 correspond to the next 7 days.

In [ ]:
df["month"] = df["ds"].dt.strftime("%B")
df.head()

In [ ]:
forecast_input = [df["y"][:-24]]
len(df["y"])

In [ ]:
dynamic_categorical_covariates = {"month": [df["month"]]}
len(df["month"])

In [20]:
# Call the forecast_with_covariates function
test_forecast, _ = tfm.forecast_with_covariates(
    forecast_input,
    dynamic_categorical_covariates=dynamic_categorical_covariates,
    xreg_mode="xreg + timesfm",
    ridge=0.0,
    force_on_cpu=False,
    normalize_xreg_target_per_input=True
)
test_forecast = np.array(test_forecast)

In [ ]:
print(f"Mean Absolute Error (MAE): {mae(test_forecast, test_df["y"])}")
print(f"Mean Squared Error (MSE): {mse(test_forecast, test_df["y"])}")
print(f"Root Mean Squared Error (RMSE): {rmse(test_forecast, test_df["y"])}")

In [22]:
forecast_input = [np.sin(np.linspace(0, 20, 512)), np.cos(np.linspace(0, 20, 512))]
dynamic_numerical_covariates = {"temperature": [np.random.rand(536), np.random.rand(536)]}
dynamic_categorical_covariates = {"weekday": [np.random.randint(0, 7, 536), np.random.randint(0, 7, 536)]}
static_numerical_covariates = {"base_price": [10.5, 15.0]}
static_categorical_covariates = {"category": ["food", "beverage"]}

In [ ]:
test_forecast, _ = tfm.forecast_with_covariates(
    forecast_input,
    dynamic_numerical_covariates=dynamic_numerical_covariates,
    dynamic_categorical_covariates=dynamic_categorical_covariates,
    static_numerical_covariates=static_numerical_covariates,
    static_categorical_covariates=static_categorical_covariates,
    freq=[0, 0],
    xreg_mode="xreg + timesfm",
    ridge=0.0,
    force_on_cpu=False,
    normalize_xreg_target_per_input=True
)

test_forecast = np.array(test_forecast)

print("Test forecast shape:", test_forecast.shape)
print("First batch test forecast first 5 values:", test_forecast[0][:5])
print("Second batch test forecast first 5 values:", test_forecast[1][:5])